In [1]:
# for image compression testing
import glob, os
import time
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# for getting screenshots
import numpy as np
from PIL import Image, ImageGrab

# for getting keyboard input
from pynput import keyboard, mouse
from pynput.keyboard import Key
from pynput.keyboard import Controller as keyController
from pynput.mouse import Controller

# makes the program DPI aware so that it can actually capture the screen correctly
# coordinates would be messed up otherwise
from ctypes import windll
user32 = windll.user32
user32.SetProcessDPIAware()

1

In [2]:
from keras.models import Model, Sequential
from keras.layers import GRU, Dense
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
# for getting keyboard/mouse input to set the bound coordinates
kb = keyController()
mouse = Controller()

In [31]:
def getTrainingData():
    training = True
    keyPresses = list()

    def on_press(key):
        if key == Key.esc:
            keyboard.Listener.stop(listener2)
            print("Lister 2 stopped!")
        if key == Key.space:
            keyPresses.pop()
            keyPresses.append(1)
    
    # Collect events until released
    listener2 = keyboard.Listener(on_press = on_press, on_release = None)
    listener2.start()
    print("Listener 2 started!")
    screenShotsTaken = 0
    images = list()
    scale = 0.05
    elapsedTime = 0    
    startTime = time.time()

    while training:                        
        im = ImageGrab.grab((x1, y1, x2, y2)) # get screenshot
        im = im.convert("L") # convert to grayscale
        width, height = im.size
        im = im.resize((round(scale * width), round(scale * height)) , Image.BICUBIC) # scale down the image
        np_im = np.array(im) / 255 # scale down the pixel values

        images.append(np_im)
        keyPresses.append(0)
        screenShotsTaken = screenShotsTaken + 1   
        training = listener2.running
        
    elapsedTime = time.time() - startTime              
    print("Total time: " + str(time.time() - startTime  ))
    print("Screenshots Taken: " + str(screenShotsTaken))
    print("Average time per screenshot: " + str(elapsedTime / screenShotsTaken))
    
    frameRate = elapsedTime / screenShotsTaken * 1000
    
    result = list()
    result.append(images)
    result.append(keyPresses)
    result.append(frameRate)
    return result

In [51]:
def playGame(model):    
    playing = True
    
    def on_press(key):
        if key == Key.esc:
            keyboard.Listener.stop(listener3)
            print("Listener 3 stopped!")
        
    # Collect events until released
    listener3 = keyboard.Listener(on_press = on_press, on_release = None)
    listener3.start()
    print("Listener 3 started!")
    
    while playing:
        im = ImageGrab.grab((x1, y1, x2, y2)) # get screenshot
        im = im.convert("L") # convert to grayscale
        width, height = im.size
        scale = 0.05
        im = im.resize((round(scale * width), round(scale * height)), Image.BICUBIC) # scale for the network
        np_im = np.array(im) / 255 # scale down the pixel values
        # print(np_im)
        # print(np.expand_dims(np_im, axis = 0).shape)
        output = model.predict(np.expand_dims(np_im, axis = 0))
        print(output)
        if output[0][0] > 0.2:
            kb.press(Key.space)
            kb.release(Key.space)
            
        playing = listener3.running

In [17]:
def on_press(key):
    global x1, y1, x2, y2
    if key == Key.shift_l:
        if x1 == 0:
            x1, y1 = mouse.position
        else:
            x2, y2 = mouse.position
    if key == Key.print_screen:
        print((y2 - y1) * 0.05)
        print((x2 - x1) * 0.05)
        # display screenshot to confirm bound coordinates
        im = ImageGrab.grab((x1, y1, x2, y2))
        plt.imshow(np.asarray(im))
        plt.title("Is this screencap right?")
        plt.show()

In [6]:
def on_release(key):
    if key == Key.ctrl_l:
        # Stop listener
        keyboard.Listener.stop(listener)
        print("Listener stopped!")
        # start training 
        print("Getting training data!")
        global arr, inputs, frameRate
        arr, inputs, frameRate = getTrainingData()
    if key == Key.tab:
        # Stop listener
        keyboard.Listener.stop(listener)
        print("listener stopped!")
        # run the network
        print("playing game!")
        playGame(model) 
    if key == Key.esc:
        # stop training
        training = False
        # stop playing
        playing = False
        # Stop listener
        keyboard.Listener.stop(listener)
        print("Listener stopped!")

In [7]:
# bound coordinates for the screenshots
x1, y1, x2, y2 = 0, 0, 0, 0
arr = list()
inputs = list()
frameRate = 0

In [50]:
# Collect events until released
listener = keyboard.Listener(on_press = on_press, on_release = on_release)
listener.start()
print("Listener started!")

Listener started!
listener stopped!
playing game!
Listener 3 started!
[[0.00752817]]
[[0.00781901]]
[[0.0076586]]
[[0.0075242]]
[[0.00876343]]
[[0.00815225]]
[[0.00795391]]
[[0.00792068]]
[[0.00729749]]
[[0.00788871]]
[[0.00787328]]
[[0.00761901]]
[[0.00748855]]
[[0.00769839]]
[[0.00776805]]
[[0.00786851]]
[[0.00823384]]
[[0.00830826]]
[[0.00835971]]
[[0.00822094]]
[[0.00812716]]
[[0.00735345]]
[[0.00743601]]
[[0.00792851]]
[[0.00654544]]
[[0.0075567]]
[[0.00765326]]
[[0.00817867]]
[[0.00736926]]
[[0.00845931]]
[[0.00887824]]
[[0.01406492]]
[[0.1123434]]
[[0.35208568]]
[[0.02047287]]
[[0.012136]]
[[0.00886411]]
[[0.00960473]]
[[0.00911507]]
[[0.04206554]]
[[0.0094648]]
[[0.0143531]]
[[0.00887957]]
[[0.02536816]]
[[0.00798969]]
[[0.008011]]
[[0.00712612]]
[[0.00668761]]
[[0.00745563]]
[[0.00809431]]
[[0.00761847]]
[[0.00747091]]
[[0.00785735]]
[[0.00886137]]
[[0.00758608]]
[[0.0074559]]
[[0.00707801]]
[[0.0105613]]
[[0.01238199]]
[[0.00690709]]
[[0.01167591]]
[[0.01007738]]
[[0.00753476

[[0.007778]]
[[0.00766693]]
[[0.00741882]]
[[0.0072105]]
[[0.00724485]]
[[0.00798939]]
[[0.00796551]]
[[0.00794548]]
[[0.00827018]]
[[0.00834942]]
[[0.0083262]]
[[0.00863871]]
[[0.00888842]]
[[0.00925551]]
[[0.00917698]]
[[0.00923104]]
[[0.00866426]]
[[0.00763819]]
[[0.00893957]]
[[0.00928804]]
[[0.00659015]]
[[0.00848424]]
[[0.00826458]]
[[0.00947734]]
[[0.00778815]]
[[0.00943245]]
[[0.01133725]]
[[0.03279067]]
[[0.15659928]]
[[0.39253512]]
[[0.16163549]]
[[0.01484307]]
[[0.00650546]]
[[0.00668272]]
[[0.00672073]]
[[0.01323923]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]
[[0.02193282]]

In [33]:
os.chdir(os.getcwd() + "/training_data")
currentTime = time.gmtime(time.time())
fileName = str(currentTime.tm_year) + str(currentTime.tm_mon)+ str(currentTime.tm_mday) + "_" + str(currentTime.tm_hour) + str(currentTime.tm_min) + str(currentTime.tm_sec)

# save inputs to text file
np.savetxt(fileName + "_" + "inputs" + ".txt", inputs, fmt = "%1.1u", header = "Key Presses")

# save numpy array of screencaps
np_arr = np.asarray(arr)
os.mkdir(fileName + "_" + "screencaps")
os.chdir(os.getcwd() + "/" + fileName + "_" + "screencaps")
for i in range(np_arr.shape[0]):
    np.savetxt("frame" + "_"  + str(i) + ".txt", np_arr[i], header = "Shape: " + str(np_arr[i].shape))
os.chdir("..")

# create and save animation
fig = plt.figure()
i = 0
frame = plt.imshow(np_arr[0], cmap = "gray", animated = True)
def update(*args):
    global i
    if (i < len(np_arr) - 1):
        i = i + 1
    else:
        i = 0
    frame.set_array(np_arr[i])
    return frame
anim = animation.FuncAnimation(fig, update, interval = frameRate, frames = len(np_arr) - 1)
anim.save(fileName + "_" + "video" + ".mp4")
plt.close()

os.chdir("..")

# create HTML preview file
HTML(anim.to_html5_video())

In [42]:
# load old data
dataToUse = "2019511_144632_"

os.chdir(os.getcwd() + "/training_data")
inputs = np.loadtxt(dataToUse + "inputs.txt")

os.chdir(os.getcwd() + "/" + dataToUse + "screencaps")
rawFrameData = list()

for i in range(len(glob.glob("*.txt"))):
    rawFrameData.append(np.loadtxt("frame_" + str(i) + ".txt"))
np_arr = np.asarray(rawFrameData)

os.chdir(os.getcwd() + "/../..")

In [21]:
# check image size
np_arr[0].shape

(31, 62)

In [52]:
model = Sequential()

In [53]:
model.add(GRU(units = 150, input_shape = (31, 62, ), return_sequences = True))

In [54]:
model.add(GRU(units = 75, return_sequences = False))

In [116]:
model.add(Dense(units = 1, activation = "sigmoid"))

In [117]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 31, 150)           95850     
_________________________________________________________________
gru_6 (GRU)                  (None, 75)                50850     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 76        
Total params: 146,776
Trainable params: 146,776
Non-trainable params: 0
_________________________________________________________________


In [118]:
model.compile(optimizer = "RMSprop", loss = "mean_squared_error", metrics = ["accuracy"])

In [43]:
model.fit(x = np_arr, y = inputs, batch_size = 32, epochs = 10, verbose = 2, validation_split = 0.1)

Train on 2882 samples, validate on 321 samples
Epoch 1/10
 - 5s - loss: 0.0452 - acc: 0.9417 - val_loss: 0.0749 - val_acc: 0.9159
Epoch 2/10
 - 5s - loss: 0.0450 - acc: 0.9421 - val_loss: 0.0745 - val_acc: 0.9159
Epoch 3/10
 - 5s - loss: 0.0439 - acc: 0.9438 - val_loss: 0.0750 - val_acc: 0.9034
Epoch 4/10
 - 5s - loss: 0.0438 - acc: 0.9459 - val_loss: 0.0730 - val_acc: 0.9128
Epoch 5/10
 - 6s - loss: 0.0433 - acc: 0.9441 - val_loss: 0.0726 - val_acc: 0.9159
Epoch 6/10
 - 5s - loss: 0.0423 - acc: 0.9434 - val_loss: 0.0741 - val_acc: 0.9034
Epoch 7/10
 - 5s - loss: 0.0423 - acc: 0.9441 - val_loss: 0.0739 - val_acc: 0.9159
Epoch 8/10
 - 5s - loss: 0.0423 - acc: 0.9427 - val_loss: 0.0754 - val_acc: 0.9159
Epoch 9/10
 - 5s - loss: 0.0428 - acc: 0.9438 - val_loss: 0.0762 - val_acc: 0.9065
Epoch 10/10
 - 5s - loss: 0.0424 - acc: 0.9448 - val_loss: 0.0751 - val_acc: 0.9128


In [112]:
# warning, saving and loading models doesn't seem to work

In [10]:
from keras.models import load_model

In [86]:
model.save(fileName + 'model')

In [30]:
model = load_model('model.h5')

In [56]:
print(model.predict(np.expand_dims(np_arr[0], axis = 0))[0][0])

-0.19757353
